<a href="https://colab.research.google.com/github/juanfranbrv/crewAI/blob/main/crewAI_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture --no-stderr
%pip install googletrans -qU
%pip install crewai==0.28.8
%pip install crewai_tools==0.1.6
# %pip install langchain_community==0.0.29
# %pip install openai -qU


In [11]:
from crewai import Agent, Task, Crew, Process
import openai

In [6]:
import os
import getpass
os.environ["OPENAI_API_KEY"] = getpass.getpass("Introduce tu OpenAI API key: ")


Introduce tu OpenAI API key: ··········


In [3]:
from googletrans import Translator

def traducir_texto(texto, lang1, lang2):
    # Crear una instancia del traductor
    traductor = Translator()

    # Traducir el texto de leng1 a leng2
    traduccion = traductor.translate(texto, src=lang1, dest=lang2)

    # Devolver el texto traducido
    return traduccion.text


In [12]:
# Step 1: Custom tools for translation and review using OpenAI API
def openai_translate(text: str, source_lang: str = "English", target_lang: str = "Spanish") -> str:
    """
    Translate text using OpenAI's API.
    """
    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": f"You are a translator from {source_lang} to {target_lang}."},
            {"role": "user", "content": f"Translate the following text: {text}"}
        ]
    )
    return response['choices'][0]['message']['content'].strip()

def openai_review(translated_text: str) -> str:
    """
    Review a translation using OpenAI's API.
    """
    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a translation reviewer. Your job is to check for errors and suggest improvements."},
            {"role": "user", "content": f"Review the following translation: {translated_text}"}
        ]
    )
    return response['choices'][0]['message']['content'].strip()

In [15]:
# Step 2: Define agents
translator = Agent(
    role='Translator',
    goal='Translate texts from English to Spanish using OpenAI.',
    backstory=(
        "You're a professional translator specializing in English to Spanish translations, "
        "with an emphasis on accurate and culturally appropriate translations."
    ),
    verbose=True,
    memory=True,
    tools=[openai_translate],
    llm_config={'model_name': 'gpt-4o-mini'} # Add llm_config
)

reviewer = Agent(
    role='Reviewer',
    goal='Review Spanish translations for errors and suggest improvements using OpenAI.',
    backstory=(
        "You're a meticulous reviewer, skilled at identifying mistakes and suggesting "
        "improvements to translations, ensuring the highest quality."
    ),
    verbose=True,
    memory=True,
    tools=[openai_review],
    llm_config={'model_name': 'gpt-4o-mini'} # Add llm_config
)

senior_editor = Agent(
    role='Senior Editor',
    goal=(
        "Oversee the translation and review process. Ensure the final translation is perfect. "
        "Terminate the process if quality is achieved or after 5 iterations."
    ),
    backstory=(
        "You're an experienced editor, responsible for ensuring translations meet quality "
        "standards. You decide when the translation is ready or when to stop iterations."
    ),
    verbose=True,
    memory=True,
    tools=[],
    llm_config={'model_name': 'gpt-4o-mini'} # Add llm_config
)

# Step 3: Define tasks
translate_task = Task(
    description="Use the OpenAI API to translate the following text from English to Spanish: {text}",
    expected_output="A translated Spanish version of the provided English text.",
    agent=translator
)

review_task = Task(
    description=(
        "Use the OpenAI API to review the Spanish translation and identify errors or areas for improvement. "
        "Provide suggestions for corrections."
    ),
    expected_output="Feedback on translation with specific corrections or suggestions.",
    agent=reviewer
)

edit_task = Task(
    description=(
        "Monitor the translation and review iterations. Ensure the translation is accurate. "
        "Stop the process if the quality is acceptable or after 5 iterations."
    ),
    expected_output="A decision on whether the translation is perfect or not.",
    agent=senior_editor)

KeyError: 'tools'

In [ ]:
# Step 4: Create the crew
crew = Crew(
    agents=[translator, reviewer, senior_editor],
    tasks=[translate_task, review_task, edit_task],
    process=Process.sequential,
    max_iterations=5
)

In [ ]:
user_text = ""

In [ ]:
# Step 5: Start the process with dynamic input
result_translate = traducir_texto(user_text,"es","eng")
result_agents = crew.kickoff(inputs={'text': user_text})
print(result)